In [0]:
# Google Cloud things

project_id = 'cs229-project-258114'
bucket_name = 'goodreads_reviews'

from google.colab import auth
auth.authenticate_user()

!gcloud config set project {project_id}

# Download the file from a given Google Cloud Storage bucket.
!gsutil cp gs://{bucket_name}/finalbooks.csv /tmp/finalbooks.csv
!gsutil cp gs://{bucket_name}/finalratings.csv /tmp/ratings.csv 
!gsutil cp gs://{bucket_name}/train.csv /tmp/train.csv 
!gsutil cp gs://{bucket_name}/test.csv /tmp/test.csv 
#!gsutil cp gs://{bucket_name}/Bayes.csv /tmp/Bayes.csv 
## !gsutil cp gs://{bucket_name}/Popularity/goodreads-best-books.zip /tmp/best.zip

# basics
import pandas as pd
import numpy as np
import collections

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# tools
import scipy
import math
import random
import sklearn
import string
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

import nltk
nltk.download('stopwords')

finalbooks = pd.read_csv('../tmp/finalbooks.csv')
ratings = pd.read_csv('../tmp/ratings.csv')
train = pd.read_csv('../tmp/train.csv')
test = pd.read_csv('../tmp/test.csv')

In [0]:
## NDGC & RMSE

def dcg_k(r, k):
    """ Discounted Cumulative Gain (DGC)  
    Args:
        r: True Ratings in Predicted Rank Order (1st element is top recommendation)
        k: Number of results to consider
    Returns:
        DCG
    """
  
    r = np.asfarray(r)[:k]
    return np.sum(2**r / np.log2(np.arange(2, r.size + 2)))      



def ndcg_k(r, k):
    """Normalized Discounted Cumulative Gain (NDCG)
    Args:
        r: True Ratings in Predicted Rank Order (1st element is top recommendation)
        k: Number of results to consider
    Returns:
        NDCG
    """
    dcg_max = dcg_k(sorted(r, reverse=True), k)
    if not dcg_max:
        return 0.
    return dcg_k(r, k) / dcg_max

def mean_ndcg(rs):
    """Mean NDCG for all users
    Args:
        rs: Iterator / For each user: True Ratings in Predicted Rank Order
    Returns:
        Mean NDCG
    """
    return np.mean([ndcg_k(r, len(r)) for r in rs])

def rmse(y,h):
    """RMSE
    Args:
        y: real y
        h: predicted y
    Returns:
        RMSE
    """
    a = y-h

    return np.sqrt(sum(a**2)/len(a))

In [0]:
## DEFINING THE TAIL
tailcomp = ratings.groupby(by= 'newbookid', as_index=False).agg({'rating':pd.Series.count}).sort_values(by = 'rating', ascending = False)
tot = sum(tailcomp['rating'])
tailcomp['popshare']= [x/tot for x in tailcomp['rating']]
tailcomp['popshare']= tailcomp['popshare'].cumsum()
tailcomp['category']= ['Head' if x<0.95 else "Tail" for x in tailcomp['popshare']]

tail = tailcomp.loc[tailcomp.popshare >= 0.95]
tail

In [0]:
def get_words(message):
    """Get the normalized list of words from a message string.

    This function should split a message into words, normalize them, and return
    the resulting list. For splitting, you should split on spaces. For normalization,
    you should convert everything to lowercase.

    Args:
        message: A string containing an SMS message

    Returns:
       The list of normalized words from the message.
    """


    words = message
    words = words.split(" ")
    words = [x.lower() for x in words]

    return words



def create_dictionary(messages):
    """Create a dictionary mapping words to integer indices.

    Args:
        messages: A list of strings containing SMS messages

    Returns:
        A python dict mapping words to integers.
    """

    word_counts = collections.defaultdict(int)

    for message in messages:
        for word in set(get_words(message)):
            word_counts[word] += 1

    resulting_dictionary = {}

    for word, count in word_counts.items():
        if count >= 10 and word not in stopwords.words('english') and len(word) > 1:
            next_index = len(resulting_dictionary)
            resulting_dictionary[word] = next_index

    return resulting_dictionary




def transform_text(messages, word_dictionary):
    """Transform a list of text messages into a numpy array for further processing.

    Args:
        messages: A list of strings where each string is an SMS message.
        word_dictionary: A python dict mapping words to integers.

    Returns:
        A numpy array marking the words present in each message.
        Where the component (i,j) is the number of occurrences of the
        j-th vocabulary word in the i-th message.
    """

    A = np.zeros((len(messages), len(word_dictionary)))

    for i, message in enumerate(messages):
        for word in get_words(message):
            if word in word_dictionary:
                A[i, word_dictionary[word]] += 1

    return A


In [0]:
finalbooks['snippet'] = finalbooks['snippet'].fillna(finalbooks['title'])
finalbooks['snippet'] = finalbooks['snippet'].str.replace(r'[^\w\s]',"")
finalbooks['snippet'] = finalbooks['snippet'].str.replace('-',"")
finalbooks['tag_cloud'] = finalbooks['tag_cloud'].str.replace('-'," ")
finalbooks['snippet'] = finalbooks['snippet'].fillna(finalbooks['tag_cloud'])
finalbooks['words'] = finalbooks['snippet'] +" "+finalbooks['tag_cloud']+" "+finalbooks['first_author']
dico = create_dictionary(finalbooks['words'])
dico

In [0]:
len(dico)

In [0]:
A = transform_text(finalbooks['words'], dico)

In [0]:
np.size(A, 0)

In [0]:
np.sum(A, axis=1).min()

In [0]:

A1 = np.sum((A>0), axis= 0)

IDF = np.log(np.size(A, 0)/A1)
IDF
len(IDF)

In [0]:
TF = A / (np.sum(A, axis=1, keepdims=True))
np.shape(TF)

In [0]:
TFiDF= TF*IDF
np.shape(TFiDF)

In [0]:
#ids = np.argsort(TFiDF.mean(axis=1))[:5]
#reverse_dictionary = {i: word for word, i in dico.items()}
#[reverse_dictionary[i] for i in ids]

In [0]:
TFiDF = TFiDF / np.sqrt((np.sum(TFiDF**2, axis = 1, keepdims=True)+0.01))

In [0]:
SimC = np.dot(TFiDF, TFiDF.T)

In [0]:
SimC

In [0]:
np.sum(SimC, axis = 1).max()

In [0]:
np.fill_diagonal(SimC, 1)

In [0]:
SimC

In [0]:
BookSim =pd.DataFrame(SimC, columns=finalbooks.title, index=finalbooks.title)

In [0]:

#np.argsort(SimC[53, :])[-5:]
finalbooks.title[np.argsort(SimC[15, :])[-6:]]
#Sim20 = pd.DataFrame(Sim20, columns=finalbooks.title[101:200], index=finalbooks.title[50:60])

In [0]:
SimC[15, [15, 2252, 6977, 4642, 2796, 1700 ]]

In [0]:
finalbooks [finalbooks.newbookid.isin(np.argsort(SimC[15, :])[-6:]+1)]

In [0]:
# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(27.5, 22.5))

# Generate a custom diverging colormap followed by the correlation heatmap
cmap =sns.diverging_palette(20, 220, n=20000)

sns.heatmap(BookSim, cmap=cmap,center = 0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [0]:
train

In [0]:
allpreds = []
train = train.sort_values(by=['newbookid'])
for i in range(15000):
  bi = train.newbookid[train.newuser_id == i+1]-1
  Simi = SimC[:, bi]
  ri = np.array(train[train.newuser_id == i+1].sort_values(by=['newbookid']).rating)
  predi = finalbooks.filter(['newbookid'])
  predi['pred'] = np.sum(Simi*ri, axis=1)/(np.sum(Simi, axis=1)+0.01)
  predi['newuser_id'] = i+1
  allpreds.append(predi)
  if (i+1)%1000 == 0: print("done: ", i+1)



In [0]:
predictions = np.concatenate(allpreds, axis=0 )

In [0]:
final =pd.DataFrame(predictions, columns=['newbookid', 'pred', 'newuser_id']) 

In [0]:
train['conc']=train['newuser_id'].map(str)+train['newbookid'].map(str)
final['conc']=final['newuser_id'].map(str)+final['newbookid'].map(str)
finalfin = final[~final.conc.isin(train.conc)]
finalfin.describe()

In [0]:
finalrank = test.merge(final,on = ['newbookid', 'newuser_id'])
finalrank = finalrank.sort_values(by=['newuser_id', 'pred'], ascending=False)
finalrank.head(5)

In [0]:
finallist = []
for i in range(15000):
    a = finalrank.loc[finalrank.newuser_id == i+1]['rating'].tolist()
    finallist.append(a)
    if (i+1)%1000 == 0: print("done: ", i+1)

In [0]:
b = np.array([ndcg_k(r, len(r)) for r in finallist])


facet, axes = plt.subplots(1, 1, figsize=(10, 3))
n, bins, patches = plt.hist(b, 200, facecolor='blue', alpha=0.5) #, log = True)   
plt.title('Distribution of NDGC among Users for the TFiDF model')
plt.show()

# [ndcg_k(r, len(r)) for r in poplista]

In [0]:
d = b[b == 1]
sum(d)/15000

In [0]:
top10 = finalfin.sort_values('pred',ascending = False).groupby('newuser_id').head(10)
top50 = finalfin.sort_values('pred',ascending = False).groupby('newuser_id').head(50)

print('(1) TF-iDF Model RMSE: ', np.round(rmse(finalrank['pred'],finalrank['rating']), decimals=3))
print('(2) TF-iDF Model NDCG: ', np.round(mean_ndcg(finallist), decimals=3))
print("(2) Median NDCG: ", np.round(np.median(b), decimals=3))
print("(2) Share of NDCG =1 among Users: ", np.round(sum(d)/15000, decimals=3))
print('(3) TF-iDF Model Div10 Score: ',np.round(sum(np.in1d(top10.newbookid, tail.newbookid))/len(top10), decimals=3))
print('(3) TF-iDF Model Div50 Score: ',np.round(sum(np.in1d(top50.newbookid, tail.newbookid))/len(top50), decimals=3))

In [0]:
finalranktrain = train.merge(final,on = ['newbookid', 'newuser_id'])
finalranktrain = finalranktrain.sort_values(by=['newuser_id', 'pred'], ascending=False)


In [0]:
finalranktrain

In [0]:
finallisttrain = []
for i in range(15000):
    a = finalranktrain.loc[finalranktrain.newuser_id == i+1]['rating'].tolist()
    finallisttrain.append(a)
    if (i+1)%1000 == 0: print("done: ", i+1)

In [0]:
print('(1) TF-iDF Train Model RMSE: ', np.round(rmse(finalranktrain['pred'],finalranktrain['rating']), decimals=3))
print('(2) TF-iDF Train Model NDCG: ', np.round(mean_ndcg(finallisttrain), decimals=3))